In [15]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs




In [41]:
url = "https://boardgamegeek.com/users?country=Vietnam&state=&city="
response = requests.get(url)
soup = bs(response.text, "html.parser")
country = soup.findAll("option")
countries = [c.text for c in country]
countries = [country.replace(" ", "%20") for country in countries]


In [25]:
#We assign weights to each country depending on the number of pages in boardgamegeek per user

def get_weight(country):
    url = f"https://boardgamegeek.com/users?country={country}&state=&city="
    # Fetch the page to get the number of pages for the country
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    last_page_link = soup.find('a', title="last page")
    if last_page_link:
            last_page_number = int(last_page_link.text.strip('[]'))
    else:
        last_page_number = 1
    
    return last_page_number
            
def createWeights(countries):
    weights = {}
    for country in countries:
        weight = get_weight(country)
        weights[country] = weight
    return weights


    


In [42]:

weights = createWeights(countries)

In [27]:
weights

{'Aruba': 15,
 'Afghanistan': 31,
 'Angola': 14,
 'Anguilla': 10,
 'Åland%20Islands': 1,
 'Albania': 30,
 'Andorra': 22,
 'United%20Arab%20Emirates': 73,
 'Argentina': 524,
 'Armenia': 25,
 'American%20Samoa': 37,
 'Antarctica': 13,
 'French%20Southern%20Territories': 1,
 'Antigua%20and%20Barbuda': 9,
 'Australia': 2129,
 'Austria': 283,
 'Azerbaijan': 10,
 'Burundi': 1,
 'Belgium': 1005,
 'Benin': 1,
 'Bonaire,%20Sint%20Eustatius%20and%20Saba': 1,
 'Burkina%20Faso': 1,
 'Bangladesh': 30,
 'Bulgaria': 149,
 'Bahrain': 14,
 'Bahamas': 14,
 'Bosnia%20and%20Herzegovina': 24,
 'Saint%20Barthélemy': 1,
 'Belarus': 80,
 'Belize': 6,
 'Bermuda': 6,
 'Bolivia,%20Plurinational%20State%20of': 1,
 'Brazil': 2130,
 'Barbados': 6,
 'Brunei%20Darussalam': 1,
 'Bhutan': 1,
 'Bouvet%20Island': 1,
 'Botswana': 1,
 'Central%20African%20Republic': 1,
 'Canada': 4744,
 'Cocos%20(Keeling)%20Islands': 1,
 'Switzerland': 316,
 'Chile': 444,
 'China': 281,
 "Côte%20d'Ivoire": 1,
 'Cameroon': 6,
 'Congo,%20The

In [5]:
import requests
from bs4 import BeautifulSoup
import random
import time

In [11]:
usernames = []

# Open a file to write usernames
with open('username2s.txt', 'w') as file:
    while len(usernames) < 10000:
        country = random.choice(countries)
        url = f"https://boardgamegeek.com/users?country={country}&state=&city=&avatar=1"
        
        # Fetch the page to get the number of pages for the country
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        last_page_link = soup.find('a', title="last page")
        if last_page_link:
            last_page_number = int(last_page_link.text.strip('[]'))
            random_page = random.randint(1, last_page_number)
            
            # Fetch a random page
            page_url = f"https://boardgamegeek.com/users/page/{random_page}?country={country}&state=&city=&avatar=1"
            response = requests.get(page_url)
            page_soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract usernames
            user_divs = page_soup.find_all('div', class_='username')
            for div in user_divs:
                username = div.find('a').text
                if username not in usernames:
                    usernames.append(username)
                    file.write(username + '\n')
                    if len(usernames) >= 10000:
                        break
        
        print("Added usernames:", len(usernames))
            
        # Wait a bit before making another request

# The 'usernames.txt' file now contains up to 1000 unique usernames.



Added usernames: 0
Added usernames: 0
Added usernames: 0
Added usernames: 25
Added usernames: 25
Added usernames: 25
Added usernames: 50
Added usernames: 50
Added usernames: 50
Added usernames: 50
Added usernames: 75
Added usernames: 75
Added usernames: 75
Added usernames: 75
Added usernames: 75
Added usernames: 75
Added usernames: 75
Added usernames: 75
Added usernames: 100
Added usernames: 125
Added usernames: 125
Added usernames: 125
Added usernames: 150
Added usernames: 150
Added usernames: 150
Added usernames: 150
Added usernames: 175
Added usernames: 175
Added usernames: 175
Added usernames: 175
Added usernames: 175
Added usernames: 175
Added usernames: 175
Added usernames: 175
Added usernames: 200
Added usernames: 200
Added usernames: 200
Added usernames: 225
Added usernames: 225
Added usernames: 225
Added usernames: 225
Added usernames: 225
Added usernames: 225
Added usernames: 225
Added usernames: 225
Added usernames: 225
Added usernames: 225
Added usernames: 225
Added usernam

In [15]:
uniques = set(usernames)
len(usernames)

10000

In [21]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import random

In [45]:
base_url = "https://boardgamegeek.com/xmlapi2/user?name=Bradsmith488&buddies=1&guilds=1"
response = requests.get(base_url)
print(response.status_code)
if response.status_code == 200:
    user = ET.fromstring(response.text)
    
    if user is not None:
        lastlogin = user.find('lastlogin').get('value')
        guilds = user.find('guilds')
        guild_total = int(guilds.get('total')) if guilds is not None else 0
        
        # Check conditions
        if (lastlogin.startswith('2023') or lastlogin.startswith('2024')) and guild_total >= 1:
            id = user.get('id')
            yearregistered = user.find('yearregistered').get('value')
            country = user.find('country').get('value')
            buddies = user.find('buddies')
            geekfriends_total = buddies.get('total') if buddies is not None else 0
            geekfriends = [b.get('name') for b in buddies.findall('buddy')] if buddies is not None else []
            guilds_list = [g.get('id') for g in guilds.findall('guild')] if guilds is not None else []
            print(id, yearregistered, country, geekfriends_total, geekfriends, guild_total, guilds_list)
            

200
932944 2014 United States 18 ['Bobdole04126623', 'Cole Wehrle', 'DCMann2', 'DeagleSeagull', 'ggambill', 'HiveGod', 'jadon79', 'jmdsplotter', 'Jonahsimmons15', 'joriswiersinga', 'kuhrusty', 'LaurenElainey', 'loftis77', 'phileklund', 'Sluggs', 'Thackerrst', 'thebadbishop', 'TheFishofDoom'] 4 ['1290', '2044', '2511', '3098']


In [49]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import random

# Example array of usernames

aux = random.sample(usernames, 150) 


# Base URL for the API (this needs to be updated to the correct API endpoint if different)

# DataFrame to hold all user data
columns = ['id', 'username', 'yearregistered', 'lastlogin', 'country', 'geekfriends', 'geekfriends_total', 'guilds', 'guild_total']
df = pd.DataFrame(columns=columns)

# Function to fetch and parse user data
def fetch_user_data(username):
    try:
        url = f"https://boardgamegeek.com/xmlapi2/user?name={username}&buddies=1&guilds=1"
        response = requests.get(url)
        if response.status_code == 200:
            user = ET.fromstring(response.content)
            if user is not None:
                lastlogin = user.find('lastlogin').get('value')
                guilds = user.find('guilds')
                guild_total = int(guilds.get('total')) if guilds is not None else 0
                
                # Check conditions
                if (lastlogin.startswith('2023') or lastlogin.startswith('2024')) and guild_total >= 1:
                    id = user.get('id')
                    yearregistered = user.find('yearregistered').get('value')
                    country = user.find('country').get('value')
                    buddies = user.find('buddies')
                    geekfriends_total = buddies.get('total') if buddies is not None else 0
                    geekfriends = [b.get('name') for b in buddies.findall('buddy')] if buddies is not None else []
                    guilds_list = [g.get('id') for g in guilds.findall('guild')] if guilds is not None else []
                    
                    return {
                        'id': id,
                        'username': username,
                        'yearregistered': yearregistered,
                        'lastlogin': lastlogin,
                        'country': country,
                        'geekfriends': geekfriends,
                        'geekfriends_total': geekfriends_total,
                        'guilds': guilds_list,
                        'guild_total': guild_total
                    }
        else:
            print(f"Failed to retrieve data for {username}, HTTP status code: {response.status_code}")
    except Exception as e:
        print(f"Error processing {username}: {e}")
    return None

data = []
# Process each username
for username in aux:
    user_data = fetch_user_data(username)
    if user_data:
        data.append(user_data)
        print('\nADDED\n')
    time.sleep(random.uniform(2,3))

df = pd.DataFrame(data, columns=columns)



# Display the DataFrame



ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED


ADDED



In [50]:
df

,id,username,yearregistered,lastlogin,country,geekfriends,geekfriends_total,guilds,guild_total
0,1842501,lucsen,2018,2023-12-21,Germany,[Robbus],1,"[742, 1303, 1622, 1648, 1999, 3344]",6
1,937901,maxdavie,2014,2024-06-21,United Kingdom,[],0,"[24, 1011]",2
2,20183,IONIAN,2004,2024-06-10,Greece,"[Aetheling, Anastasoulis, Asghan, deschuyter, ...",28,"[119, 1303]",2
3,81964,gusberman,2006,2024-02-29,Argentina,"[Arcangelurbano, gonzalezm, jamesdavis, marioa...",6,"[1037, 1303]",2
4,2606378,Voxith,2020,2024-06-20,France,[],0,"[1028, 4143]",2
5,274443,Louminous,2009,2024-06-21,Greece,"[Anemelos, Atactoulis, dontbecruel, Emil 109, ...",10,"[119, 2378, 2714, 3578]",4
6,132272,duartec,2007,2024-06-24,Portugal,"[Abruk, Admeto, airjudden, Alphawolf, Anonimus...",115,"[128, 437, 505, 909, 1081, 1190, 1500, 2044, 3...",10
7,64326,Casulo,2005,2024-06-18,Brazil,"[acaporal, Anderson Depizol, BezouroErratico, ...",35,"[673, 1622, 2044, 3260]",4
8,307487,Jack Spirio,2010,2024-06-24,Austria,"[FrAnkX0r88, frotes, Matthias_K, mephistophele...",6,"[1438, 2246]",2
9,2829277,Oicalyptus,2021,2024-06-24,Germany,"[Arradelle, birdman284, CarolineBlack, elizhar...",13,"[1303, 2246]",2


In [51]:
df.to_csv('players.csv', index=False)